In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import re
import numpy as np
import pandas as pd
import math
import pickle
from collections import Counter
import unicodedata
Stopwords = set(stopwords.words('english'))
ps=PorterStemmer()

In [2]:
temp=open('pickles/posting_list.pkl',"rb")
posting_lists=pickle.load(temp)

temp=open('pickles/file_idx.pkl','rb')
file_index=pickle.load(temp)

In [3]:
unique_words=set(posting_lists.keys())

In [4]:
def remove_special_characters(text):
    regex = re.compile('[^a-zA-Z\s]')
    text_returned = re.sub(regex,' ',text)
    return text_returned
def strip_accents(text):
    return ''.join(c for c in unicodedata.normalize('NFKD', text) if unicodedata.category(c) != 'Mn')


In [5]:
data=pd.read_csv('../English.csv')

In [6]:
def brs(query,counter=10):
    query=strip_accents(query)
    text = remove_special_characters(query)
    text = re.sub(re.compile('\d'),'',text)
    words = word_tokenize(text)
    words = [word.lower() for word in words]
    words = [ps.stem(word) for word in words]
    words = [word for word in words if word not in Stopwords]
    if len(words)>1:
        query_words=[words[0]]
    else:
        query_words=words

    for i in range(1,len(words)):
        if words[i] not in ["and","or"]:
            if query_words[-1] not in ["and","or"]:
                query_words.append("and")
                query_words.append(words[i])
            else:
                query_words.append(words[i])
        elif query_words[-1] not in ["and","or"]:
            query_words.append(words[i])
    
    operators=[]
    main_words=[]

    for w in query_words:
        if w.lower() in ["and","or"]:
            operators.append(w.lower())
        else:
            main_words.append(w)
    n=len(file_index)
    word_vector=[]
    word_vector_matrix=[]

    for w in main_words:
        word_vector=[0]*n
        if w in unique_words:
            for x in posting_lists[w].keys():
                word_vector[x]=1
        word_vector_matrix.append(word_vector)
    for w in operators:
        vector1=word_vector_matrix[0]
        vector2=word_vector_matrix[1]

        if w=="and":
            result=[b1&b2 for b1,b2 in zip(vector1,vector2)]

            word_vector_matrix.pop(0)
            word_vector_matrix.pop(0)

            word_vector_matrix.insert(0,result)
        else:
            result=[b1|b2 for b1,b2 in zip(vector1,vector2)]

            word_vector_matrix.pop(0)
            word_vector_matrix.pop(0)

            word_vector_matrix.insert(0,result)
    final_word_vector=word_vector_matrix[0]
    slokes=[]
    description=[]
    verse_index=[]
    index=0
    for i in final_word_vector:
        if not counter:
            break
        if i==1:
            verse_index.append(index)
            counter=counter-1
        index+=1
    return verse_index

In [7]:
verse_index=brs('Bhagavad-gītā is the widely read theistic')

In [8]:
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
0,1,1,धृतराष्ट्र उवाचधर्मक्षेत्रे कुरुक्षेत्रे समवेत...,Bhagavad-gītā is the widely read theistic scie...
